言語モデルのファインチューニング　2025/06/19

*   最後のcell[]が動かず

# 第13章 RAG

## 13.3 RAG 向けに LLM を指示チューニングする

### 13.3.1 AI 王データセットを用いた指示チューニング

#### 環境の準備

In [1]:
!pip install datasets transformers[torch,sentencepiece] trl peft bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers.trainer_utils import set_seed

# 乱数のシードを設定
set_seed(42)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### データセットの準備

In [3]:
from datasets import load_dataset

# Hugging Face Hubのllm-book/aio-retrieverのリポジトリから
# AI王データセットを読み込む
dataset = load_dataset(
    "llm-book/aio-retriever", trust_remote_code=True
)

# 読み込まれたデータセットの形式と事例数を確認
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.56k [00:00<?, ?B/s]

aio-retriever.py:   0%|          | 0.00/3.58k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 22335
    })
    validation: Dataset({
        features: ['qid', 'competition', 'timestamp', 'section', 'number', 'original_question', 'original_answer', 'original_additional_info', 'question', 'answers', 'passages', 'positive_passage_indices', 'negative_passage_indices'],
        num_rows: 1000
    })
})


In [5]:
from pprint import pprint

# 読み込まれたデータセットの内容を確認
pprint(dataset["validation"][0])

{'answers': ['ジェット団'],
 'competition': '第2回AI王',
 'negative_passage_indices': [1,
                              2,
                              4,
                              6,
                              8,
                              9,
                              10,
                              11,
                              13,
                              14,
                              15,
                              16,
                              17,
                              18,
                              19,
                              20,
                              21,
                              23,
                              24,
                              25,
                              26,
                              27,
                              28,
                              29,
                              30,
                              31,
                              32,
                              33,
     

In [6]:
from typing import Any

def filter_example(
    example: dict[str, Any], max_passages: int = 3
) -> bool:
    """上位max_passages件のパッセージに正例が含まれていない事例を除外"""
    if len(example["positive_passage_indices"]) == 0:
        return False
    if example["positive_passage_indices"][0] >= max_passages:
        return False

    return True

dataset = dataset.filter(filter_example)

Filter:   0%|          | 0/22335 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [7]:
def process_example(
    example: dict[str, Any], max_passages: int = 3
) -> dict[str, Any]:
    """質問、パッセージ、正解の組からプロンプトを作成し、会話データに変換"""

    # exampleから必要な情報を取得
    question = example["question"]
    answer = example["answers"][0]
    passages = [p["text"] for p in example["passages"]]

    # max_passages件のパッセージを選択
    passages = passages[:max_passages]

    messages: list[dict[str, str]] = []
    # プロンプトとパッセージをユーザのメッセージとして会話データに追加
    prompt_text = "".join(
        [
            "あなたには今からクイズに答えてもらいます。",
            "問題を与えますので、その解答のみを簡潔に出力してください。\n",
            "また解答の参考になりうるテキストを与えます。",
            "解答を含まない場合もあるのでその場合は無視してください。\n\n",
            "---\n",
            "\n\n".join(passages),
            "\n---\n\n",
            f"問題: {question}",
        ]
    )
    messages.append({"role": "user", "content": prompt_text})
    # LLMが出力すべき内容（クイズ問題の答え）を会話データに追加
    messages.append({"role": "assistant", "content": answer})

    # 会話データを事例の"messages"フィールドに追加
    example["messages"] = messages
    return example

dataset = dataset.map(
    process_example, remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/13951 [00:00<?, ? examples/s]

Map:   0%|          | 0/637 [00:00<?, ? examples/s]

In [8]:
# 前処理後のデータセットの形式と事例数を確認
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 13951
    })
    validation: Dataset({
        features: ['messages'],
        num_rows: 637
    })
})


In [9]:
# 前処理後のデータセットの内容を確認
pprint(dataset["validation"][0])

{'messages': [{'content': 'あなたには今からクイズに答えてもらいます。問題を与えますので、その解答のみを簡潔に出力してください。\n'
                          'また解答の参考になりうるテキストを与えます。解答を含まない場合もあるのでその場合は無視してください。\n'
                          '\n'
                          '---\n'
                          'ニューヨークのウエスト・サイド。午後5時。ポーランド系アメリカ人の少年非行グループ「ジェッツ」(ジェット団)と、新参のプエルトリコ系アメリカ人の少年非行グループ「シャークス」(シャーク団)は、なわばりを巡って対立している。今日も2グループの間で争いが起きるが警官の呼子笛の音に止められる(“Prologue”「プロローグ」)。クラプキ巡査とシュランク警部補が現れて少年たちに説教をして帰っていく。ジェッツのリーダー・リフはシャークスとの関係をはっきりさせるために決闘しようと言い出し、ジェッツのメンバーが賛成する。ついては決闘についての取り決めをシャークスとする必要があり、リフは自分の副官にトニーを選ぶ。メンバーは初めトニーはもう抜けたと反対するが、リフは(海兵隊のように)「一度ジェッツになったら死ぬまでジェッツだ」と歌う。\n'
                          '\n'
                          '『ウエストサイド物語』(ウエストサイドものがたり)は、宝塚歌劇団によるミュージカル作品。ブロードウェイ・ミュージカルの傑作『ウエストサイド物語』の日本での上演の一つである。\n'
                          '\n'
                          '『ウエスト・サイド物語』(ウエスト・サイドものがたり、West Side '
                          'Story)は、アーサー・ローレンツ脚本、レナード・バーンスタイン音楽、スティーヴン・ソンドハイム歌詞のブロードウェイ・ミュージカル。原案ジェローム・ロビンズ。1957年初演。『ウエスト・サイド・ストーリー』とも

#### トークナイザとモデルの準備

In [10]:
!pip install bitsandbytes

In [ ]:
!pip install -U bitsandbytes

In [11]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

# Hugging Face Hubにおけるモデル名を指定
base_model_name = "llm-book/Swallow-7b-hf-oasst1-21k-ja"

# モデルの量子化の設定
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4ビット量子化のパラメータを読み込む
    bnb_4bit_quant_type="nf4",  # NF4量子化を使用
    bnb_4bit_compute_dtype=torch.bfloat16,  # 計算時のデータ型としてBF16を使用
)

# モデルの量子化の設定を用いてモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,  # 量子化設定
    use_cache=False,  # 後にgradient checkpointingを有効にするために必要
    device_map="auto",
)

# トークナイザを読み込む
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

#### 指示チューニング前のモデルの評価

In [12]:
from datasets import Dataset
from tqdm.notebook import tqdm
from transformers import PreTrainedModel

def evaluate(
    model: PreTrainedModel, dataset: Dataset
) -> tuple[list[str], list[str], float]:
    """データセットの各問題に対するモデルの出力を評価し、正解率を算出"""
    pred_answers = []
    gold_answers = []
    num_correct = 0

    for example in tqdm(dataset):
        # プロンプトにチャットテンプレートを適用
        model_inputs = tokenizer.apply_chat_template(
            example["messages"][:-1],
            add_generation_prompt=True,
            return_tensors="pt",
        ).to("cuda")

        # プロンプトの長さ（トークン数）を取得しておく
        input_length = model_inputs.shape[1]

        # モデルにプロンプトを入力し、出力を得る
        generated_ids = model.generate(
            model_inputs,
            max_new_tokens=32,
            do_sample=False,
            temperature=None,
            top_p=None,
        )

        # モデルの出力から答えの部分を文字列として取り出す
        pred_answer = tokenizer.batch_decode(
            generated_ids[:, input_length:], skip_special_tokens=True
        )[0]

        # 正解の文字列を取り出す
        gold_answer = example["messages"][-1]["content"]

        # モデルの答えと正解が一致していれば正答とカウント
        if pred_answer == gold_answer:
            num_correct += 1

        # モデルの答えと正解をそれぞれリストに追加
        pred_answers.append(pred_answer)
        gold_answers.append(gold_answer)

    # 正解率を計算
    accuracy = num_correct / len(pred_answers)

    return pred_answers, gold_answers, accuracy

In [13]:
# 指示チューニング前のモデルを使って評価
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"]
)

# 無料版ColabのT4 GPUなどでは評価に時間を要するため、最初の100事例のみで評価
# pred_answers, gold_answers, accuracy = evaluate(
#     model, dataset["validation"].take(100)
# )

print(f"正解率: {accuracy:.1%}")

  0%|          | 0/637 [00:00<?, ?it/s]

正解率: 52.1%


In [14]:
# モデルが予測した答えを表示
for pred_answer, gold_answer in zip(
    pred_answers[:20], gold_answers[:20]
):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")

正解: ジェット団 / 予測: ジェッツ
正解: コマイ / 予測: スケトウダラ
正解: START / 予測: START
正解: ニュートン / 予測: アイザック・ニュートン
正解: 天平文化 / 予測: 天平文化
正解: アメリカンリーグ / 予測: アメリカンリーグ
正解: 華道 / 予測: 池坊、草月流、小原流は、日本の伝統的な生け花の三大流派である。池坊は1
正解: ラストベルト / 予測: ラストベルト
正解: 天童市 / 予測: 天童市
正解: 医学部 / 予測: 安部公房は東京大学医学部出身。
正解: 村田珠光 / 予測: 山上宗二は、「侘び茶」の創始者として知られる室町時代の茶人である。彼は
正解: 23時 / 予測: 日本のテレビ業界で「プライムタイム」といえば、毎日19時から23時までの時間帯のことです。
正解: 佐々木彩夏 / 予測: 佐々木彩夏
正解: 早口言葉 / 予測: 英語で「タングツイスター」という言葉遊びは「早口言葉」です。
正解: 昭和基地 / 予測: 昭和基地
正解: 開口一番 / 予測: 「開口一番」
正解: マクベス / 予測: マクベス
正解: ニ長調 / 予測: ト短調
正解: 版籍奉還 / 予測: 版籍奉還
正解: IBS / 予測: IBS


#### 指示チューニングの準備

In [15]:
# 訓練セットのすべての事例にチャットテンプレートを適用
tokenized_train_dataset = [
    tokenizer.apply_chat_template(example["messages"])
    for example in dataset["train"]
]

In [16]:
from trl import DataCollatorForCompletionOnlyLM

# collate関数を初期化
bos = tokenizer.bos_token
collator = DataCollatorForCompletionOnlyLM(
    # ユーザとアシスタントそれぞれの発話開始文字列
    instruction_template=bos + "ユーザ：",
    response_template=bos + "アシスタント：",
    tokenizer=tokenizer,  # トークナイザ
)

In [17]:
from peft import LoraConfig, TaskType, get_peft_model

# LoRAの設定
peft_config = LoraConfig(
    r=128,  # 差分行列のランク
    lora_alpha=128,  # LoRA層の出力のスケールを調整するハイパーパラメータ
    lora_dropout=0.05,  # LoRA層に適用するドロップアウト
    task_type=TaskType.CAUSAL_LM,  # LLMが解くタスクのタイプを指定
    # LoRAで学習するモジュール
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
)

model.enable_input_require_grads()  # 学習を行うために必要
model = get_peft_model(model, peft_config)  # モデルにLoRAを適用
model.print_trainable_parameters()  # 学習可能なパラメータ数を表示

trainable params: 319,815,680 || all params: 7,149,785,088 || trainable%: 4.4731


#### 指示チューニングの実行

In [18]:
from transformers import Trainer, TrainingArguments

# 訓練のハイパーパラメータを設定
training_args = TrainingArguments(
    output_dir="./drive/MyDrive/llm_book/RAG_IT_results",  # 結果の保存フォルダ
    bf16=True,  # BF16を使用した学習の有効化
    max_steps=100,  # 訓練ステップ数
    per_device_train_batch_size=2,  # 訓練時のバッチサイズ
    gradient_accumulation_steps=8,  # 勾配累積のステップ数（5.5.2節）
    gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3節）
    optim="paged_adamw_8bit",  # 最適化器
    learning_rate=1e-4,  # 学習率
    lr_scheduler_type="cosine",  # 学習率スケジューラの種類
    max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3節）
    warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8節）
    logging_steps=10,  # ロギングの頻度
    save_steps=50,  # モデルの保存頻度
    report_to="none",  # 外部ツールへのログを無効化
)

# 無料版のT4 GPUなど、低メモリ環境での学習パラメータ
# バッチサイズと勾配累積のステップ数を小さく設定
# training_args = TrainingArguments(
#     output_dir="./drive/MyDrive/llm_book/RAG_IT_results",  # 結果の保存フォルダ
#     bf16=True,  # BF16を使用した学習の有効化
#     max_steps=100,  # 訓練ステップ数
#     per_device_train_batch_size=1,  # 訓練時のバッチサイズ
#     gradient_accumulation_steps=4,  # 勾配累積のステップ数（5.5.2節）
#     gradient_checkpointing=True,  # 勾配チェックポインティングの有効化（5.5.3節）
#     optim="paged_adamw_8bit",  # 最適化器
#     learning_rate=1e-4,  # 学習率
#     lr_scheduler_type="cosine",  # 学習率スケジューラの種類
#     max_grad_norm=0.3,  # 勾配クリッピングにおけるノルムの最大値（9.4.3節）
#     warmup_ratio=0.1,  # 学習率のウォームアップの長さ（5.2.8節）
#     logging_steps=10,  # ロギングの頻度
#     save_steps=50,  # モデルの保存頻度
#     report_to="none",  # 外部ツールへのログを無効化
# )

# Trainerを初期化
trainer = Trainer(
    model,
    train_dataset=tokenized_train_dataset,  # トークンID化されたデータセット
    data_collator=collator,  # ラベルの加工及びミニバッチ構築処理を行うモジュール
    args=training_args,  # 訓練の設定
    tokenizer=tokenizer,  # パラメータ保存時にトークナイザも一緒に保存するために指定
)

# モデルの訓練を実行
trainer.train()

/tmp/ipython-input-18-1705970447.py:41: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,0.375600
20,0.169000
30,0.093800
40,0.144400
50,0.127600
60,0.124100
70,0.121100
80,0.165800
90,0.123400
100,0.124500


TrainOutput(global_step=100, training_loss=0.1569329434633255, metrics={'train_runtime': 541.4705, 'train_samples_per_second': 2.955, 'train_steps_per_second': 0.185, 'total_flos': 5.182191345962189e+16, 'train_loss': 0.1569329434633255, 'epoch': 0.11467889908256881})

#### 指示チューニング後のモデルの評価

In [19]:
# 指示チューニング後のモデルを使って評価
pred_answers, gold_answers, accuracy = evaluate(
    model, dataset["validation"]
)

# 無料版ColabのT4 GPUなどでは評価に時間を要するため、最初の100事例のみで評価
# pred_answers, gold_answers, accuracy = evaluate(
#     model, dataset["validation"].take(100)
# )

print(f"正解率: {accuracy:.1%}")

  0%|          | 0/637 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


正解率: 82.3%


In [20]:
# モデルが予測した解答を表示
for pred_answer, gold_answer in zip(
    pred_answers[:20], gold_answers[:20]
):
    print(f"正解: {gold_answer} / 予測: {pred_answer}")

正解: ジェット団 / 予測: ジェッツ
正解: コマイ / 予測: スケトウダラ
正解: START / 予測: START
正解: ニュートン / 予測: アイザック・ニュートン
正解: 天平文化 / 予測: 天平文化
正解: アメリカンリーグ / 予測: アメリカンリーグ
正解: 華道 / 予測: いけばな
正解: ラストベルト / 予測: ラストベルト
正解: 天童市 / 予測: 天童市
正解: 医学部 / 予測: 医学部
正解: 村田珠光 / 予測: 村田珠光
正解: 23時 / 予測: 23時
正解: 佐々木彩夏 / 予測: 玉井詩織
正解: 早口言葉 / 予測: 早口言葉
正解: 昭和基地 / 予測: 昭和基地
正解: 開口一番 / 予測: 開口一番
正解: マクベス / 予測: マクベス
正解: ニ長調 / 予測: ヘ長調
正解: 版籍奉還 / 予測: 版籍奉還
正解: IBS / 予測: IBS


#### モデルの保存

In [21]:
from huggingface_hub import notebook_login

# Hugging Face Hubにログイン
notebook_login()

In [ ]:
# 無料版Colab（T4 GPU）の場合はRAMの制限で量子化前のモデルを読み込めない場合があります
# その場合は以下のコードで、学習後のLoRAパラメータのみをアップロードすることが可能です
# model.push_to_hub("singletongue/Swallow-7b-hf-oasst1-21k-ja-aio-retriever")

In [22]:
from peft import PeftModel

# 学習したLoRAのパラメータを量子化していない学習前のモデルに足し合わせる
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
)
checkpoint_path = "./drive/MyDrive/llm_book/RAG_IT_results/checkpoint-100"
tuned_model = PeftModel.from_pretrained(base_model, checkpoint_path)

# LoRAのパラメータのみをアップロードする場合は次の行をコメントアウト
tuned_model = tuned_model.merge_and_unload()

# Hugging Face Hubのリポジトリ名を指定
# "YOUR-ACCOUNT"は自らのユーザ名に置き換えてください
repo_name = "YOUR-ACCOUNT/Swallow-7b-hf-oasst1-21k-ja-aio-retriever"

# トークナイザをアップロード
tokenizer.push_to_hub(repo_name)
# モデルをアップロード
tuned_model.push_to_hub(repo_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-68538f0d-0510944f504dbff11a1895dc;7b8dff18-cf22-4d5c-b5be-cdce78eb6c64)

Invalid username or password.